### NGOI with jonathan priority algo

In [3]:
import numpy as np
import pandas as pd
import random
from numba import jit

In [2]:
data=pd.read_excel('Complete_BR_instances.xlsx')
data=data.drop('Unnamed: 0',axis=1)
#reading the dataframe with all the instances

In [3]:
def priority_generator(df,file,inst,difficulty):
    
    df=df[df['Txt_file']==file]
    df=df[df['Problem_instance']==inst]
    box_types = list(range(1,df['Box_type'].max()+1))
    len_box_types = len(box_types)
    group_of_boxes = []
    for i in range(0,difficulty):
        set_of_boxes = []
        parts = int(np.ceil(len_box_types/difficulty))
        if parts >= len(box_types):
            parts = len(box_types)

        for i in range(0,parts):
            box = random.randint(0,len(box_types)-1)
            set_of_boxes.append(box_types[box])
            box_types.pop(box)

        group_of_boxes.append(set_of_boxes)
    df_priority=df
    df_priority['load_priority'] = 0
    for e,group in enumerate(group_of_boxes):
        for box_type in group:
            df_priority.loc[df_priority.Box_type == box_type, 'load_priority'] = (e+1)
                
    return df_priority,len(df_priority.load_priority.unique())

In [4]:
def Empty_space(L,W):
    return(pd.DataFrame(data=0,index=[L],columns=[W]))


In [5]:
def Obj_Space(xcor,ycor,l,w,L,W,objNo):
    if (l+xcor>L) or (w+ycor>W):
        print('object doesnt fit in space')
    else:
        temp=pd.DataFrame(data=0,index=np.unique([xcor,(l+xcor),L]), columns=np.unique([ycor,(w+ycor),W]))
        temp.loc[(l+xcor),(w+ycor)]=objNo
        if 0 in temp.columns:
            temp=temp.drop(columns=0)
        if 0 in temp.index:
            temp=temp.drop(index=0)
        return(temp)

In [6]:
def Merge(space,obj):
    x12=np.sort(np.unique(np.concatenate((obj.columns.values,space.columns.values),axis=None)))
    y12=np.sort(np.unique(np.concatenate((obj.index.values,space.index.values),axis=None)))
    
    pd_space=pd.DataFrame(data=80,index=y12,columns=x12)
    pd_obj=pd.DataFrame(data=80,index=y12,columns=x12)
    pd_final=pd.DataFrame(data=80,index=y12,columns=x12)
    
    for i in space.index.values:
        for j in space.columns.values:
            pd_space.loc[i,j]=space.loc[i,j]
    
    for i in range(len(pd_space.index)):
        for j in range(len(pd_space.columns)):
            t1=i
            while pd_space.iloc[t1,j]==80 and t1+1<len(pd_space.index):
                t1=t1+1
            pd_space.iloc[i,j]=pd_space.iloc[t1,j]
      
    
    for i in range(len(pd_space.index)):
        for j in range(len(pd_space.columns)):
            t2=j
            while pd_space.iloc[i,t2]==80 and t2+1<len(pd_space.columns):
                t2=t2+1
            pd_space.iloc[i,j]=pd_space.iloc[i,t2]
       
    for i in range(len(pd_space.index)):
        for j in range(len(pd_space.columns)):
            if pd_space.iloc[i,j]==80:
                pd_space.iloc[i,j]=0
    
    
    for i in obj.index.values:
        for j in obj.columns.values:
            pd_obj.loc[i,j]=obj.loc[i,j]
    
    
    for i in range(len(pd_obj.index)):
        for j in range(len(pd_obj.columns)):
            t1=i
            while pd_obj.iloc[t1,j]==80 and t1+1<len(pd_obj.index):
                t1=t1+1
            pd_obj.iloc[i,j]=pd_obj.iloc[t1,j]
      
    
    for i in range(len(pd_obj.index)):
        for j in range(len(pd_obj.columns)):
            t2=j
            while pd_obj.iloc[i,t2]==80 and t2+1<len(pd_obj.columns):
                t2=t2+1
            pd_obj.iloc[i,j]=pd_obj.iloc[i,t2]
       
    for i in range(len(pd_obj.index)):
        for j in range(len(pd_obj.columns)):
            if pd_obj.iloc[i,j]==80:
                pd_obj.iloc[i,j]=0
    
   
    for i in range(len(pd_final.index)):
        for j in range(len(pd_final.columns)):
            pd_final.iloc[i,j]=pd_space.iloc[i,j]+pd_obj.iloc[i,j]
    
     
    return(pd_final)
    
    


In [4]:
@jit(nopython=True) #calling numba
def number_gen(x):
    y=np.array([80,237,1])
    temp=2
    
    while y.size<x:
        temp2=0
        for i in y:
            for j in y:
                if (i+j) == temp:
                    temp2=1
        
        if temp2==0:
            y=np.append(y,temp)
        temp=temp+1
        
    return(y)

In [5]:
lib=number_gen(100)

In [6]:
lib

array([ 80, 237,   1,   3,   5,   7,   9,  11,  13,  15,  17,  19,  21,
        23,  25,  27,  29,  31,  33,  35,  37,  39,  41,  43,  45,  47,
        49,  51,  53,  55,  57,  59,  61,  63,  65,  67,  69,  71,  73,
        75,  77,  79, 161, 163, 165, 167, 169, 171, 173, 175, 177, 179,
       181, 183, 185, 187, 189, 191, 193, 195, 197, 199, 201, 203, 205,
       207, 209, 211, 213, 215, 217, 219, 221, 223, 225, 227, 229, 231,
       233, 235, 237, 239, 320, 401, 403, 405, 407, 409, 411, 413, 415,
       417, 419, 421, 423, 425, 427, 429, 431, 433], dtype=int64)

In [10]:
def Spacefinder(pngoi):
    
    if sum(sum(pngoi.values==0))==0:
        return(-1,-1,-1,-1,-1,-1)

    for i in range(len(pngoi.index)):
        
        for j in range(len(pngoi.columns)):
            t_ng_sp_cnt_col=1
            t_ng_sp_cnt_row=1
            while i+t_ng_sp_cnt_row<=len(pngoi.index) and pngoi.iloc[i+t_ng_sp_cnt_row-1,j] == 0:

                t_ng_sp_cnt_row+=1

                #print(i+t_ng_sp_cnt_row-1,j)
            
            ngoi_t1=0
            while t_ng_sp_cnt_row>1 and j+t_ng_sp_cnt_col<=(len(pngoi.columns))and ngoi_t1==0:
                
                
                #print(i,i+t_ng_sp_cnt_row-1,j+t_ng_sp_cnt_col-1,j+t_ng_sp_cnt_col)
                if sum(sum(pngoi.iloc[i:i+t_ng_sp_cnt_row-1,j+t_ng_sp_cnt_col-1:j+t_ng_sp_cnt_col].values))==0:
                    t_ng_sp_cnt_col+=1
                else:
                    ngoi_t1=1
                
            if t_ng_sp_cnt_row-1 + t_ng_sp_cnt_col-1>0:
                #print(pngoi)
                #print(i,j,t_ng_sp_cnt_row-1,t_ng_sp_cnt_col-1)
                if i==0:
                    strt_sp_x=0
                else: strt_sp_x=pngoi.index[i-1]
                
                if j==0:
                    strt_sp_y=0
                else: strt_sp_y=pngoi.columns[j-1]
                    
                if i+t_ng_sp_cnt_row-1>=len(pngoi.index):
                    stp_sp_x=pngoi.index[i+t_ng_sp_cnt_row-2]
                else:
                    stp_sp_x=pngoi.index[i+t_ng_sp_cnt_row-1]
                    
                if j+t_ng_sp_cnt_col-1>=len(pngoi.columns):
                    stp_sp_y=pngoi.columns[j+t_ng_sp_cnt_col-2]
                else:
                    stp_sp_y=pngoi.columns[j+t_ng_sp_cnt_col-2]
                        
                #print(pngoi)
                #print(strt_sp_x,strt_sp_y,stp_sp_x,stp_sp_y,i,j,i+t_ng_sp_cnt_row-1,j+t_ng_sp_cnt_col-1)
                
                return(strt_sp_x,strt_sp_y,stp_sp_x,stp_sp_y,i,j)

In [11]:

volu_util=pd.DataFrame(columns=['File_No.', 'Instance_No.','Volume_util','No_of _prioritygroups', 'No of errors'])
##declaring dataframe to store the volume utilisation values

for ii in range(10):
    for jj in range(100):
        #going thorugh all the instances and files
        priority_number=random.randint(1,10)
        [sel_data,priority_number]=priority_generator(data,'br'+str(ii+1)+'.txt',jj+1,priority_number)
#opening each instance
        No_of_items = len(sel_data)
#loading number of box types in instance

        quant=np.array(sel_data.Number_of_boxes.values)
        total_item=sel_data.Number_of_boxes.sum()

        l=np.array(sel_data.Box_length.values)
        w=np.array(sel_data.Box_width.values)
        h=np.array(sel_data.Box_height.values)
        
        priority=np.array(sel_data.load_priority.values)
        
        initial_coordinate=[0,0,0]
        
        # inital coordinate of the box placed
        

        delta=np.zeros(6)
        lcap=np.zeros((No_of_items,6))
        wcap=np.zeros((No_of_items,6))
        hcap=np.zeros((No_of_items,6))

        # delta is the orientation and lcap, wcap and hcap are the dimension of the box for the selected orientation

        X= sel_data.Container_length.values[0]
        Y= sel_data.Container_width.values[0]
        Z= sel_data.Container_height.values[0]
        # loading the dimensions of the container



        for i in range(No_of_items):
            lcap[i,0]=l[i]
            lcap[i,1]=w[i]
            lcap[i,2]=l[i]
            lcap[i,3]=h[i]
            lcap[i,4]=w[i]
            lcap[i,5]=h[i]
            wcap[i,0]=w[i]
            wcap[i,1]=l[i]
            wcap[i,2]=h[i]
            wcap[i,3]=w[i]
            wcap[i,4]=h[i]
            wcap[i,5]=l[i]
            hcap[i,0]=h[i]
            hcap[i,1]=h[i]
            hcap[i,2]=w[i]
            hcap[i,3]=l[i]
            hcap[i,4]=l[i]
            hcap[i,5]=w[i]

        # all orientation of each type of item for each orientation type

        Volume=np.zeros(No_of_items)
        for i in range(No_of_items):
            Volume[i]=l[i]*w[i]*h[i]
#volume of ith item

        placement=pd.DataFrame(columns=['item_no','delta','init_x','init_y', 'init_z', 'lcap', 'wcap', 'hcap'])
    #item type, orientation, position x,y,z & dimension in x,y,z due to orientation
        loading_area=pd.DataFrame(columns=['x', 'y', 'z','Xcap', 'Ycap', 'Zcap'])
    #loading area is the current considered place to fill position x,y,z with the maximum length,width and height possible.
        loading_area.loc[0]=np.array([0,0,0,X,Y,Z])

        #initialize ngoi matrix space
        p_ngoi=Empty_space(X,Y)
        i_ngoi=2
        error_ngoi=0
        palette_org=pd.DataFrame(columns=['Pal_No','Obj_no','x_cor','y_cor','length','width','left_frnt_x','left_frnt_y','filled'])
        
        
        
        # initializing placement positions and loading area dataframe

        k=0
        p=0
        stop=0
        id_x=id_y=0
        x_sp_1=x_sp_2=y_sp_1=y_sp_2=-1
        # counter k for selected objects
        # counter p for selected box for the bottom position(may skip)
        # stop is termination counter when the code cant move to a better loading area.

        while sum(quant)>0 and stop==0:#while any box remaining and stop counter =0

            rank=pd.DataFrame(columns=['item_no','delta','a','b','c','d'])
            
            minpriority=np.inf
            for i in range(No_of_items):
                if quant[i]>0:
                    minpriority=min(minpriority,priority[i])
            #dataframe to create ranking of boxes
            d=0
            #counter for ranking
            for i in range(No_of_items):
                for j in range(6):
                    
                    if (lcap[i][j]<=loading_area.loc[k,'Xcap']) and wcap[i][j]<=(loading_area.loc[k,'Ycap']) and hcap[i][j]<=(loading_area.loc[k,'Zcap']) and quant[i]>0 and priority[i]<=minpriority:
            #does the item fit in loading area and is available                   
                        rank.loc[d,'item_no']=i
                        rank.loc[d,'delta']=j
                        rank.loc[d,'a']=int(min((loading_area.loc[k,'Zcap']/hcap[i][j]),quant[i]))*Volume[i] #max(min(Total available height/ box height, available boxes)* volume of boxes) 
                        rank.loc[d,'b']=loading_area.loc[k,'Xcap']+lcap[i][j] # boxes with min length
                        rank.loc[d,'c']=Volume[i] #boxes with maximum volume individually
                        rank.loc[d,'d']=wcap[i][j] #boxes with minimum width
                        d=d+1
        #trying all possible combinations if possible

            if rank.item_no.count()>0:
                rank1=pd.DataFrame(rank[rank['a']==max(rank.a)])
                if rank1.item_no.count()>1:
                    rank1=pd.DataFrame(rank1[rank1['b']==min(rank1.b)])
                    if rank1.item_no.count()>1:
                        rank1=pd.DataFrame(rank1[rank1['c']==max(rank1.c)])
                        if rank1.item_no.count()>1:
                            rank1=pd.DataFrame(rank1[rank1['d']==min(rank1.d)])
                            if rank1.item_no.count()>1:
                                rank1[rank1.index==rank1.index.min()]
                                #creating ranking as per the 4 rules and selecting a box
                placement.loc[k,'item_no']=rank1.item_no.iloc[0]
                placement.loc[k,'delta']=rank1.delta.iloc[0]
                placement.loc[k,'init_x']=loading_area.loc[k,'x']
                placement.loc[k,'init_y']=loading_area.loc[k,'y']
                placement.loc[k,'init_z']=loading_area.loc[k,'z']
                placement.loc[k,'lcap']=lcap[placement.loc[k]['item_no']][placement.loc[k]['delta']]
                placement.loc[k,'wcap']=wcap[placement.loc[k]['item_no']][placement.loc[k]['delta']]
                placement.loc[k,'hcap']=hcap[placement.loc[k]['item_no']][placement.loc[k]['delta']]
                placement.loc[k,'palette_no']=p
                placement.loc[k,'palette_x']=placement[placement['palette_no']==p].iloc[0].init_x
                placement.loc[k,'palette_y']=placement[placement['palette_no']==p].iloc[0].init_y
                placement.loc[k,'palette_z']=placement[placement['palette_no']==p].iloc[0].init_z
                #placing the box and updating the value in dataframe
                
                #adding ngoi matrix updation
                if placement.loc[k,'init_z']==0:
                    q_ngoi=Obj_Space(placement.loc[k,'init_x'],placement.loc[k,'init_y'],placement.loc[k,'lcap'],placement.loc[k,'wcap'],X,Y,lib[i_ngoi])
                    
                    finaltest = Merge(p_ngoi,q_ngoi)
                    p_ngoi=finaltest
                    palette_org.loc[i_ngoi-2]=[i_ngoi,lib[i_ngoi],placement.loc[k,'init_x'],placement.loc[k,'init_y'],placement.loc[k,'lcap'],placement.loc[k,'wcap'],(placement.loc[k,'init_x']+placement.loc[k,'lcap']),placement.loc[k,'init_y'],0]
                    i_ngoi=i_ngoi +1
                
                
                
                #check overlap
                for icheck in np.unique(p_ngoi.values):
                    if icheck not in lib and icheck!=0:
                        error_ngoi=error_ngoi+1
                
                
                #placement matrix for the output boxes
                loading_area.loc[k+1,'Xcap']= placement.loc[k,'lcap']
                loading_area.loc[k+1,'Ycap']= placement.loc[k,'wcap']
                loading_area.loc[k+1,'Zcap']= loading_area.loc[k,'Zcap']-placement.loc[k,'hcap']
                loading_area.loc[k+1,'x']= loading_area.loc[k,'x']
                loading_area.loc[k+1,'y']= loading_area.loc[k,'y']
                loading_area.loc[k+1,'z']= loading_area.loc[k,'z'] + placement.loc[k,'hcap']
                # changing the reference point for next iteration in z direction that is height
                quant[placement.loc[k,'item_no']]=quant[placement.loc[k,'item_no']]-1
                #updating remaining items
                k=k+1
                #updating counter for the next item

            else:

#if items cant be fit on top
                if placement[placement['palette_no']==p].item_no.count()>0:
                    
                    
                    [x_sp_1,y_sp_1,x_sp_2,y_sp_2,id_x,id_y]=Spacefinder(p_ngoi)
                    if x_sp_1!=-1:
                        
                        loading_area.loc[k+1,'x']= x_sp_1
                        loading_area.loc[k+1,'y']= y_sp_1
                        loading_area.loc[k+1,'z']= 0
                        loading_area.loc[k+1,'Xcap']= x_sp_2-x_sp_1
                        loading_area.loc[k+1,'Ycap']= y_sp_2-y_sp_1
                        loading_area.loc[k+1,'Zcap']= Z-loading_area.loc[k+1,'z']
                        k=k+1
                                            #move it to the next location on the right
                        p=p+1
                    
                else:
                    
                    if loading_area.loc[k,'x']==x_sp_1 and loading_area.loc[k,'y']==y_sp_1:
                        p_ngoi.iloc[id_x,id_y]=237
                     
                    [x_sp_1,y_sp_1,x_sp_2,y_sp_2,id_x,id_y]=Spacefinder(p_ngoi)
                    if x_sp_1!=-1:
                        loading_area.loc[k+1,'x']= x_sp_1
                        loading_area.loc[k+1,'y']= y_sp_1
                        loading_area.loc[k+1,'z']= 0
                        loading_area.loc[k+1,'Xcap']= x_sp_2-x_sp_1
                        loading_area.loc[k+1,'Ycap']= y_sp_2-y_sp_1
                        loading_area.loc[k+1,'Zcap']= Z-loading_area.loc[k+1,'z']
                        k=k+1
                        
                    
#if cant be fit on the right even then move front with the maximum of the location of a box and length combined
                    #loading_area.loc[k+1,'x']= max(placement.init_x + placement.lcap)
                    #loading_area.loc[k+1,'y']= 0
                    #loading_area.loc[k+1,'z']= 0
                    #loading_area.loc[k+1,'Xcap']= X-loading_area.loc[k+1,'x']
                    #loading_area.loc[k+1,'Ycap']= Y
                    #loading_area.loc[k+1,'Zcap']= Z
                    #k=k+1
# if the next loading is outside or repeating the loading area then increment stop counter
                    if sum(sum(p_ngoi.values==0))==0:
                        stop=1
#storing the result
        placement.to_excel('bischoff1.5.3\\output'+str(ii+1)+'.'+str(jj+1)+'.xlsx')
        ##Formatting the output as per visualisation requirement
        outxx=placement[['init_y','init_z','init_x','wcap','hcap','lcap']]
        outxx=outxx.rename(columns ={'init_y':'X','init_z':'Y','init_x':'Z','wcap':'Length','hcap':'Height','lcap':'Width'} , inplace = False)
        outxx.to_excel('bischoff1.5.3\\display_output.xlsx'+str(ii+1)+'.'+str(jj+1)+'.xlsx',index=False)
        #Important the output excel file is that you need for your visulisation to run. i have checked it
        #storing the volume utilisation
        kl=0
        for i in range(No_of_items):
            kl=kl+Volume[i]*placement[placement.item_no==i].item_no.count()
            

        volu_util.loc[(ii*100)+jj]=[ii+1,jj+1,kl/(X*Y*Z),priority_number,error_ngoi]
        

In [12]:
volu_util.to_excel('bischoff1.5.3\\volume_utilisation_1.6.xlsx')

In [13]:
volu_util.Volume_util.mean()

0.7505975523785283

In [14]:
volu_util['No of errors'].max()

0.0

In [15]:
volu_util.Volume_util.min()

0.585791910964645

In [16]:
volu_util.Volume_util.max()

0.9071432606992046